In [1]:
val columns = spark.read.parquet("s3://agimodeltrainer/Clean_Data/ColumnMap.parquet").as[(String, Seq[String])].collect.toMap
val finalTable = spark.read.parquet("s3://agimodeltrainer/Clean_Data/FinalTable.parquet")
finalTable.printSchema

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1600108946459_0005,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

columns: scala.collection.immutable.Map[String,Seq[String]] = Map(Census_Business_Patterns -> List(FIPS, Year, Maps), Complex_Race -> List(FIPS, Year, One_Race, Only_White, Only_Black, Only_Native, Only_Cherokee, Only_Chippewa, Only_Navajo, Only_Sioux, Only_Asian, Only_Indian, Only_Chinese, Only_Filipino, Only_Japanese, Only_Korean, Only_Vietnamese, Only_Asian_Other, Only_Pacific, Only_Hawaiian, Only_Guamanian, Only_Samoan, Only_Pacific_Other, Only_Other, Twoplus_Races, White_Black, White_Native, White_Asian, Black_Native), Simple_Race -> List(FIPS, Year, White, Black, Native, Asian, Pacific, Other), Hispanic -> List(FIPS, Year, Hispanic, Mexican, Puerto_Rican, Cuban, Hispanic_Other, Not_Hispanic), Educational_Attainment -> List(FIPS, Year, Pop_18_24, Less_HS_Grad_18_24, HS_Grad_18_24, ...finalTable: org.apache.spark.sql.DataFrame = [FIPS: string, Year: int ... 71 more fields]
root
 |-- FIPS: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Population: integer (nullab

In [2]:
import org.apache.spark.sql.{Row, DataFrame}
// Really just a quantile calculator right now, but leaves room for additional custom functions
def myEval(table:DataFrame, agg:String, col:String) = {
    val columnArray = table.select(col).orderBy(col).map(_.get(0).toString).collect
    val percent = agg.substring(0, agg.indexOf('%')).toDouble/100.0
    val index = Math.round((columnArray.length-1) * percent).toInt
    Row(columnArray(index))
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{Row, DataFrame}
myEval: (table: org.apache.spark.sql.DataFrame, agg: String, col: String)org.apache.spark.sql.Row


In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._

// Create summary table like same-named built in Spark function. It runs slower but otherwise seems fine
def customSummary(table:DataFrame, columns:Seq[String], aggs:Seq[String]) = {
    val newTable = new Array[Array[String]](aggs.size)
    var newSchema = List(StructField("", StringType, false))
    for ((col, i) <- columns.zipWithIndex) {
        newSchema = newSchema :+ StructField(col, StringType, false)
    }
    
    for ((agg, i) <- aggs.zipWithIndex) {
        newTable(i) = new Array[String](columns.size+1)
        newTable(i)(0) = agg.replace("!", "")
        val flag = !agg.contains('!')
        for ((col, j) <- columns.zipWithIndex) {
            val temp1 = if (flag) {
                table.select(expr(s"${agg}(${col})")).collect.head
            } else {
                myEval(table, agg.replace("!", ""), col)
            }
            newTable(i)(j+1) = try {
                temp1.getInt(0).toString
            } catch {
                case e: ClassCastException => try {
                    temp1.getLong(0).toString
                } catch {
                    case e: ClassCastException => try {
                        f"${temp1.getDouble(0)}%1.2f"
                    } catch {
                        case e: ClassCastException => temp1.getString(0)
                    }
                }
            }
        }
    }
    val t = newTable.toSeq.map(Row(_:_*))
    val someDF = spark.createDataFrame(
        spark.sparkContext.parallelize(t),
        StructType(newSchema)
    )
    someDF.show
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql._
import org.apache.spark.sql.types._
customSummary: (table: org.apache.spark.sql.DataFrame, columns: Seq[String], aggs: Seq[String])Unit


In [4]:
val selected = columns("Simple_Race")//.diff(Seq("FIPS", "Year"))
finalTable.select(selected.head, selected.tail: _*).summary().show

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

selected: Seq[String] = List(FIPS, Year, White, Black, Native, Asian, Pacific, Other)
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|summary|              FIPS|              Year|            White|             Black|           Native|             Asian|            Pacific|             Other|
+-------+------------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|  count|             21972|             21972|            21972|             21972|            21972|             21972|              21972|             21972|
|   mean| 30393.87233752048|2014.0003640997634|85.61506007646088| 9.718728381576552|2.883783906790462|1.6029127981066897|0.17578281449117472|2.2876661205170272|
| stddev|15153.499264321712|1.9998861772627188| 16.3791392499106|14.657830440780335|8.109682410891063| 3.1925

In [5]:
customSummary(
    finalTable,
    selected, 
    Seq("count", "mean", "stddev", "min", "!25%", "!50%", "!75%", "max")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-------+------+-----+------+-----+-------+-----+
|      |    FIPS|   Year| White|Black|Native|Asian|Pacific|Other|
+------+--------+-------+------+-----+------+-----+-------+-----+
| count|   21972|  21972| 21972|21972| 21972|21972|  21972|21972|
|  mean|30393.87|2014.00| 85.62| 9.72|  2.88| 1.60|   0.18| 2.29|
|stddev|15153.50|   2.00| 16.38|14.66|  8.11| 3.19|   1.02| 3.75|
|   min|   01001|   2011|  3.80| 0.00|  0.00| 0.00|   0.00| 0.00|
|   25%|   18179|   2012|  80.0|  1.0|   0.6|  0.4|    0.0|  0.4|
|   50%|   29177|   2014|  92.4|  2.9|   1.0|  0.8|    0.1|  1.0|
|   75%|   45081|   2016|  97.1| 11.5|   1.9|  1.6|    0.2|  2.5|
|   max|   56045|   2017|100.00|87.20| 96.00|61.70|  34.30|58.80|
+------+--------+-------+------+-----+------+-----+-------+-----+



In [6]:
def unpackNAICSMap(table:DataFrame, mapCol:String, naics:Seq[Int], size:Seq[String]) = {
    var newTable = table
    for (naic <- naics) {
        newTable = newTable.withColumn(naic.toString, col(mapCol)(naic))
        for ((s,i) <- size.zipWithIndex) {
            if (s != null) {
                newTable = newTable.withColumn(naic + ":" + s, col(naic.toString)(i)).na.fill(0)
            }                                  
        }
        newTable = newTable.drop(naic.toString)
    }
    newTable
}
def makeSizeSeq(sizes:Seq[String]) = {
    val masterSizes = Seq("Total", "1_4", "5_9", "10_19", "20_49", "50_99", "100_249", "250_499", "500_999", "1000_1499", "1500_2499", "2500_4999", "5000plus")
    var outSizes = Seq[String]()
    for (s <- masterSizes) {
        if (sizes.contains(s)) 
            outSizes = outSizes :+ s
        else
            outSizes = outSizes :+ null
    }
    outSizes
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

unpackNAICSMap: (table: org.apache.spark.sql.DataFrame, mapCol: String, naics: Seq[Int], size: Seq[String])org.apache.spark.sql.DataFrame
makeSizeSeq: (sizes: Seq[String])Seq[String]


In [8]:
//TODO: Correlation chart
// Perform correlations between each column
// Similar to summary, save the comparing column as the left most element
// each following element is the correlation between column (leftside) and column (above)
// Will probably have to save that dataframe to disk, visualize in python

def myCorr(table:DataFrame) = {
    val cols = table.columns
    var alreadyVisited = Seq[String]()
    
    var newSchema = List(StructField("", StringType, false))
    for (col <- cols) {
        newSchema = newSchema :+ StructField(col, DoubleType, true)
    }
        
    var newTableAsSeq = Seq[Seq[Any]]()
    for (a <- cols) {
        var rowAsSeq = Seq[Any](a)
        for (b <- cols) {
            if (alreadyVisited.contains(b)) {
                rowAsSeq = rowAsSeq :+ null
            } else {
                try {
                    rowAsSeq = rowAsSeq :+ table.select(corr(a, b)).head.getDouble(0)
                } catch {case e:NullPointerException => 
                    rowAsSeq = rowAsSeq :+ null
                }
            }
        }
        newTableAsSeq = newTableAsSeq :+ rowAsSeq
        alreadyVisited = alreadyVisited :+ a
    }
    
    val t = newTableAsSeq.map(Row(_:_*))
    val someDF = spark.createDataFrame(
        spark.sparkContext.parallelize(t),
        StructType(newSchema)
    )
    someDF
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

myCorr: (table: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [9]:
val naics = Seq(0, 11, 21, 22, 23, 31, 42, 44, 48, 51, 52, 53, 54, 55, 56, 61, 62, 71, 72, 81, 92)
val sizes = makeSizeSeq(Seq("1_4", "5_9", "10_19", "20_49", "50_99", "100_249", "250_499", "500_999", "1000_1499", "1500_2499", "2500_4999", "5000plus"))
val naicsSizes = for {n <- naics; s <- sizes; if (s != null)} yield n + ":" + s
val unpack = "Year" +: naicsSizes
val test2 = unpackNAICSMap(finalTable, "Maps", naics, sizes).select("FIPS", unpack: _*)
test2.printSchema
val finaltest2 = finalTable.
    drop("Maps").
    join(test2, Seq("FIPS", "Year"), "inner")
finaltest2.printSchema

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

naics: Seq[Int] = List(0, 11, 21, 22, 23, 31, 42, 44, 48, 51, 52, 53, 54, 55, 56, 61, 62, 71, 72, 81, 92)
sizes: Seq[String] = List(null, 1_4, 5_9, 10_19, 20_49, 50_99, 100_249, 250_499, 500_999, 1000_1499, 1500_2499, 2500_4999, 5000plus)
naicsSizes: Seq[String] = List(0:1_4, 0:5_9, 0:10_19, 0:20_49, 0:50_99, 0:100_249, 0:250_499, 0:500_999, 0:1000_1499, 0:1500_2499, 0:2500_4999, 0:5000plus, 11:1_4, 11:5_9, 11:10_19, 11:20_49, 11:50_99, 11:100_249, 11:250_499, 11:500_999, 11:1000_1499, 11:1500_2499, 11:2500_4999, 11:5000plus, 21:1_4, 21:5_9, 21:10_19, 21:20_49, 21:50_99, 21:100_249, 21:250_499, 21:500_999, 21:1000_1499, 21:1500_2499, 21:2500_4999, 21:5000plus, 22:1_4, 22:5_9, 22:10_19, 22:20_49, 22:50_99, 22:100_249, 22:250_499, 22:500_999, 22:1000_1499, 22:1500_2499, 22:2500_4999, 22:5000plus, 23:1_4, 23:5_9, 23:10_19, 23:20_49, 23:50_99, 23:100_249, 23:250_499, 23:500_999, 23:1000_1499, 23:1500_2499, 23:2500_4999, 23:5000plus, 31:1_4, 31:5_9, 31:10_19, 31:20_49, 31:50_99, 31:100_249,

In [10]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val assembler = new VectorAssembler().
  setInputCols(finaltest2.drop("FIPS").drop("Year").columns).
  setOutputCol("features")
val computeCorrelationOfMe = assembler.transform(finaltest2.drop("FIPS").drop("Year")).select("features")
val Row(coeff1: Matrix) = Correlation.corr(computeCorrelationOfMe, "features").head

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{Matrix, Vectors}
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_849b34e7dd5c
computeCorrelationOfMe: org.apache.spark.sql.DataFrame = [features: vector]
coeff1: org.apache.spark.ml.linalg.Matrix =
1.0                     -0.10962190297515613    ... (322 total)
-0.10962190297515613    1.0                     ...
0.10961880124660892     -0.999999799620711      ...
0.08675043031117285     -0.1310735909776316     ...
0.0431916265161201      -0.1637603435619562     ...
0.013623595460029204    -0.17605289924896428    ...
0.028890251254482788    -0.12002776526170887    ...
0.09693970035504364     0.04915861202987477     ...
0.26767654534019863     0.26991213662548197     ...
0.1998669744914176      0.19398985151798        ...
-0.023462809580194675   0.06538028061957725     ...
-0.15099021298289605    0.03172780

In [ ]:
import org.apache.spark.ml.linalg.Matrices
val matrix = Matrices.dense(3,3, Array(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0))
val cols = Seq("A","B", "C")
var newSchema = List(StructField("", StringType, false))
for (col <- cols) {
    newSchema = newSchema :+ StructField(col, DoubleType, true)
}
var newTableAsSeq = Seq[Seq[Any]]()
for ((row, col) <- matrix.rowIter.zip(cols.iterator)) {
    newTableAsSeq = newTableAsSeq :+ (Seq(col) ++ row.toArray.toSeq)
}
val t = newTableAsSeq.map(Row(_:_*))
val someDF = spark.createDataFrame(
    spark.sparkContext.parallelize(t),
    StructType(newSchema)
)

In [ ]:
def matrixToDF(mat:Matrix, cols:Array[String]) = {
    
    var newSchema = List[StructField](StructField("label", StringType, false))
    for (col <- cols) {
        newSchema = newSchema :+ StructField(col, DoubleType, true)
    }
    
        
    var newTableAsSeq = Seq[Seq[Any]]()
    for ((row, col) <- mat.rowIter.zip(cols.iterator)) {
        newTableAsSeq = newTableAsSeq :+ (Seq(col) ++ row.toArray.toSeq)
    }
    
    val t = newTableAsSeq.map(Row(_:_*))
    val someDF = spark.createDataFrame(
        spark.sparkContext.parallelize(t),
        StructType(newSchema)
    )
    someDF
}

In [ ]:
val corrMatrix = matrixToDF(coeff1, finaltest2.drop("FIPS").drop("Year").columns)

In [ ]:
import org.apache.spark.sql.functions.udf
def viewCorr(corrMatrix:DataFrame, features:Seq[String]) = {
    val fil = udf((label:String) => features.contains(label))
    corrMatrix.select("label", features.sorted:_*).filter(fil($"label")).orderBy("label").show(features.size, false)
}

In [ ]:
viewCorr(corrMatrix, Seq("Total_AGI", "Bachelor_18_24", "Black", "22:1000_1499"))